# 匯入 SHP file

In [21]:
import shapefile # pip isntall pyshp
sf = shapefile.Reader("../mapdata202010230200/TOWN_MOI_1091016.shp", encoding = 'utf-8')

display(f'{len(sf)} features in SHP file')

town_list = [x.__geo_interface__ for x in sf]

display('Features in town_list looks like this:')
display(town_list[0])    
    


'368 features in SHP file'

'Features in town_list looks like this:'

{'type': 'Feature',
 'properties': {'TOWNID': 'V02',
  'TOWNCODE': '10014020',
  'COUNTYNAME': '臺東縣',
  'TOWNNAME': '成功鎮',
  'TOWNENG': 'Chenggong Township',
  'COUNTYID': 'V',
  'COUNTYCODE': '10014'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[(121.40981573700003, 23.21369278500009),
    (121.40984267700003, 23.213661019000085),
    (121.40987962400004, 23.21361611900005),
    (121.40988182900003, 23.213613440000074),
    (121.409963973, 23.213513608000085),
    (121.4097828680001, 23.213464187000056),
    (121.40970386900005, 23.213536825000062),
    (121.40963608700008, 23.213641573000075),
    (121.40965519500003, 23.21375052700006),
    (121.40963705600007, 23.213855154000044),
    (121.40945973100008, 23.213845504000062),
    (121.40942544900008, 23.213940952000087),
    (121.40932155500002, 23.21401708600007),
    (121.40923562700004, 23.213930159000085),
    (121.40931459600006, 23.213847227000088),
    (121.40927818800003, 23.213746263000075),
    (121.40930504000005

# 建立 DB 

In [39]:
import sqlite3

# this command will create a db file if the db file doesn't exist, otherwise connect to the exsited db.
conn = sqlite3.connect('practica.db')

# disconnect from the db.
conn.close()

# 在 DB 中建立 table 用來存放 features

In [40]:
import sqlite3
sql_create = f'''
    CREATE TABLE town_features(
        towncode TEXT NOT NULL UNIQUE,
        feature TEXT NOT NULL
        );
    '''
conn = sqlite3.connect('practica.db')
conn.execute(sql_create)
conn.commit()
conn.close()

# 把 features 存到 table 中

In [41]:
import json
conn = sqlite3.connect('practica.db')
try:
    for feature in town_list:
        towncode = feature['properties']['TOWNCODE']    
        feature_json =  json.dumps(feature, ensure_ascii = False)
        conn.execute('insert into town_features (towncode, feature) values (?,?)', [towncode,feature_json] )
        conn.commit()
except Exception as e:
    print(e)
    
finally:
    conn.close()

# 觀察 table 內的資料

In [42]:
import pandas as pd
conn = sqlite3.connect('practica.db')
df = pd.read_sql('select * from town_features' , conn)
conn.close()
df

towncode                                            feature
0    10014020  {"type": "Feature", "properties": {"TOWNID": "...
1    10013210  {"type": "Feature", "properties": {"TOWNID": "...
2    10009130  {"type": "Feature", "properties": {"TOWNID": "...
3    10014110  {"type": "Feature", "properties": {"TOWNID": "...
4    10014160  {"type": "Feature", "properties": {"TOWNID": "...
..        ...                                                ...
363  10013040  {"type": "Feature", "properties": {"TOWNID": "...
364  66000020  {"type": "Feature", "properties": {"TOWNID": "...
365  66000270  {"type": "Feature", "properties": {"TOWNID": "...
366  10015120  {"type": "Feature", "properties": {"TOWNID": "...
367  10015020  {"type": "Feature", "properties": {"TOWNID": "...

[368 rows x 2 columns]

In [48]:
conn = sqlite3.connect('practica.db')
towncode = '10015020'
(feature,) = conn.execute('select feature from town_features where towncode = ?' , (towncode, ) ).fetchone()
conn.close()
display(feature)

'{"type": "Feature", "properties": {"TOWNID": "U07", "TOWNCODE": "10015020", "COUNTYNAME": "花蓮縣", "TOWNNAME": "鳳林鎮", "TOWNENG": "Fenglin Township", "COUNTYID": "U", "COUNTYCODE": "10015"}, "geometry": {"type": "Polygon", "coordinates": [[[121.52611348700009, 23.817322921000084], [121.52627530900008, 23.817290008000043], [121.52633855200008, 23.81725743000004], [121.52636360600002, 23.81718085600005], [121.52642650400003, 23.81706412400007], [121.52650359200004, 23.81697323900005], [121.52654536500006, 23.816856590000043], [121.52659320100008, 23.81650040100004], [121.52659955700005, 23.816332078000073], [121.52659889400002, 23.816170260000035], [121.52646169600007, 23.81533571500006], [121.52624492100006, 23.814192222000088], [121.52606558600007, 23.813383715000043], [121.5258789720001, 23.81251697400006], [121.5257396830001, 23.81142434900005], [121.52561592000006, 23.81042793800009], [121.52555639800005, 23.809644909000042], [121.52548992600009, 23.80908286600004], [121.5253954380000

# 你的工作
1. 在同一個db中建立一個叫做'menu'的table，有兩個欄位：'name' & 'content'，資料型態都是text，並且不能是空值，其中'name'是不可重複的。
2. 用上次的方式產生鄉鎮市區目錄，並存入 'menu' 這個table中。
3. 在Flask server 之中把menu從db中取出來，並透過網頁顯示。